In [ ]:
###add those code before your training loop
import subprocess  #past to your code
import threading  #past to your code
import time
import pickle

def compute_average_hourly_utilization(minute_utilizations):
    padding_length = (60 - len(minute_utilizations) % 60) % 60
    # Pad the utilization list with zeros
    padded_utilizations = minute_utilizations + [0] * padding_length
    
    num_hours = len(padded_utilizations) // 60
    
    # Compute the hourly averages
    hourly_averages = [sum(padded_utilizations[i*60:(i+1)*60])/60.0 for i in range(num_hours)]

    return hourly_averages

def get_gpu_utilization(device_id):
    try:
        result = subprocess.check_output(
            ["nvidia-smi", "--query-gpu=utilization.gpu", "--format=csv,noheader,nounits"]
        ).decode("utf-8").strip().split('\n')
        return int(result[device_id])############!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    except Exception as e:
        print(f"Error querying GPU utilization: {e}")
        return None

# Function to repeatedly track GPU utilization every minute
def track_gpu_utilization_periodically(device_id):
    if not stop_tracking:
        utilization = get_gpu_utilization(device_id)
        if utilization is not None:
            utilization_list.append(utilization)
        threading.Timer(60, track_gpu_utilization_periodically, args=[device_id]).start()

# ... [rest of the PyTorch code for model, optimizer, etc.]

utilization_list = []
stop_tracking = False

# Start the periodic GPU tracking
track_gpu_utilization_periodically(device_id)
start_time = time.time()

###################################################################### before training


# training loop

###################################################################### after training
stop_tracking = True
end_time = time.time()
total_time = end_time - start_time

# Print GPU utilizations
print("***********************************************")
print("energy results")
print("\nGPU Utilizations recorded every minute:", utilization_list)

with open('utilization_data.pkl', 'wb') as f:
    pickle.dump(utilization_list, f)
    
average_hourly_utilizations = compute_average_hourly_utilization(utilization_list)
print("Average hourly GPU Utilizations:", average_hourly_utilizations)
print(f"\nTotal training time: {total_time:.2f} seconds.")
